# Pipeline SDK

In [1]:
import kfp

@kfp.dsl.pipeline(name='my-pipeline_1207-0027', description='Hello How do you do')
def hello_pipeline():
    train_task = hello_component()

@kfp.dsl.component
def hello_component():
    return kfp.dsl.ContainerOp(
        name='my-ContainerOp',
        image='docker.io/kubeflowkatib/mxnet-mnist:v0.13.0-rc.1'
    )    

# Pipeline 컴파일

In [2]:
kfp.compiler.Compiler().compile(hello_pipeline, 'my_pipeline.zip')

/opt/conda/lib/python3.8/site-packages/kfp/dsl/_container_op.py:1257: FutureWarning: Please create reusable components instead of constructing ContainerOp instances directly. Reusable components are shareable, portable and have compatibility and support guarantees. Please see the documentation: https://www.kubeflow.org/docs/pipelines/sdk/component-development/#writing-your-component-definition-file The components can be created manually (or, in case of python, using kfp.components.create_component_from_func or func_to_container_op) and then loaded using kfp.components.load_component_from_file, load_component_from_uri or load_component_from_text: https://kubeflow-pipelines.readthedocs.io/en/stable/source/kfp.components.html#kfp.components.load_component_from_file
  warnings.warn(


In [3]:
! ls -l my_pipeline.zip

-rw-r--r-- 1 jovyan users 573 Feb 21 09:44 my_pipeline.zip


In [4]:
! unzip -l my_pipeline.zip

Archive:  my_pipeline.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
     1040  1980-01-01 00:00   pipeline.yaml
---------                     -------
     1040                     1 file


In [5]:
! unzip -p my_pipeline.zip

apiVersion: argoproj.io/v1alpha1
kind: Workflow
metadata:
  generateName: my-pipeline-1207-0027-
  annotations: {pipelines.kubeflow.org/kfp_sdk_version: 1.8.11, pipelines.kubeflow.org/pipeline_compilation_time: '2022-02-21T09:44:36.481194',
    pipelines.kubeflow.org/pipeline_spec: '{"description": "Hello How do you do",
      "name": "my-pipeline_1207-0027"}'}
  labels: {pipelines.kubeflow.org/kfp_sdk_version: 1.8.11}
spec:
  entrypoint: my-pipeline-1207-0027
  templates:
  - name: my-containerop
    container: {image: 'docker.io/kubeflowkatib/mxnet-mnist:v0.13.0-rc.1'}
    metadata:
      labels:
        pipelines.kubeflow.org/kfp_sdk_version: 1.8.11
        pipelines.kubeflow.org/pipeline-sdk-type: kfp
        pipelines.kubeflow.org/enable_caching: "true"
      annotations: {pipelines.kubeflow.org/component_spec: '{"name": "Hello component"}'}
  - name: my-pipeline-1207-0027
    dag:
      tasks:
      - {name: my-containerop, template: my-containerop}
  arguments:
    parameters: [

# Pipeline Experiment

In [6]:
client = kfp.Client()

my_experiment = client.create_experiment(
    name='My first Experiment' # Experiment name
)

# Pipeline 실행

In [7]:
my_run = client.run_pipeline(
    my_experiment.id, 
    'oh_my_experiment', # Run name
    'my_pipeline.zip'
)